In [ ]:
#-----------------------------------------------------------------------------------------------------
# Connection to pyVMMonitor for profiling
#-----------------------------------------------------------------------------------------------------
import sys
sys.path.append('D:\\Program Files\\Brainwy\\PyVmMonitor 1.1.2\\public_api')
import pyvmmonitor
pyvmmonitor.connect()

In [5]:
# Libraries Imports
#-----------------------------------------------------------------------------------------------------
from configparser import ConfigParser
from sqlalchemy import create_engine
from datetime import datetime
import pandas as pd
import numpy as np
from datetime import date, timedelta

# Load connexion configuration file
#-----------------------------------------------------------------------------------------------------
config_filename = "./conf/GetWeather.ini"

# Read config.ini file
config_object = ConfigParser()
config_object.read(config_filename)
if  not config_object:
    print("Error while loading configuration !!!")

# Log initialization
cfg = config_object['INFO']
# Establish connection with DB (using sqlalchemy)
#-----------------------------------------------------------------------------------------------------
db_connection_str = "mysql+pymysql://{}:{}@{}/WeatherDB".format(cfg['user'],cfg['password'],cfg['host'])
db_connection = create_engine(db_connection_str)

In [4]:
#----------------------------------------------------------------------------------
# Table : RawRecords
# Query : List records created by locations and creation dates
#----------------------------------------------------------------------------------
raw_data = pd.read_sql(
"""SELECT date(date_timestamp) AS dte, Locations.name, count(date_timestamp) AS nb_records
    FROM RawRecords
    INNER JOIN Locations ON Locations.id = RawRecords.id_location
    WHERE id_location IN (SELECT id FROM Locations)
    GROUP BY id_location, date(date_timestamp) 
    ORDER BY date(date_timestamp);""",
    con=db_connection
)
pdata = pd.DataFrame(raw_data)
pdata.head(100)

,dte,name,nb_records
0,2021-06-10,Lausanne,87
1,2021-06-10,Sion,70
2,2021-06-10,Evolène,36
3,2021-06-10,Hérémence,95
4,2021-06-10,Geneva,82
...,...,...,...
95,2021-08-23,Geneva,47
96,2021-08-23,Sion,36
97,2021-08-23,Hérémence,39
98,2021-08-23,Lausanne,39


In [ ]:
#----------------------------------------------------------------------------------
# Table : RawRecords
# Query : List records created by locations and creation dates grouped 
#         by locations + month
#----------------------------------------------------------------------------------
raw_data = pd.read_sql(
""" SELECT date(date_timestamp) AS dte, Locations.name AS LName, count(date_timestamp) AS nb_records
    FROM RawRecords
    INNER JOIN Locations ON Locations.id = RawRecords.id_location
    WHERE MONTH(DATE(date_timestamp)) in (SELECT MONTH(DATE(date_timestamp)) FROM RawRecords GROUP BY MONTH(DATE(date_timestamp)))
    AND ( id_location IN (SELECT id FROM Locations) ) 
    GROUP BY id_location, month(date_timestamp) 
    ORDER BY id_location, month(date_timestamp);""",
    con=db_connection  
)
pdata = pd.DataFrame(raw_data)
pdata.head(100)

In [5]:
#----------------------------------------------------------------------------------
# Table : RawRecords
# Query : Count all records
#----------------------------------------------------------------------------------
raw_data = pd.read_sql(
""" SELECT COUNT(*) AS totalRecs 
    FROM RawRecords;""",
    con=db_connection
)
pdata = pd.DataFrame(raw_data)
pdata.head(10)

,totalRecs
0,92507


In [ ]:
#----------------------------------------------------------------------------------
# Table : RawRecords
# Query : Count sorted number of records per locations
#----------------------------------------------------------------------------------
raw_data = pd.read_sql(
    """ SELECT Locations.name as name, COUNT(*) as nbRecords
        FROM RawRecords
        INNER JOIN Locations ON RawRecords.id_location = Locations.id
        GROUP BY id_location
        ORDER BY nbRecords DESC;""", 
        con=db_connection
)
pdata = pd.DataFrame(raw_data)
pdata.head(10)

In [6]:
#----------------------------------------------------------------------------------
# Table : RawRecords
# List records created by locations and creation dates
#----------------------------------------------------------------------------------
raw_data = pd.read_sql(
""" SELECT date(date_timestamp) AS dte, Locations.name, count(date_timestamp) AS nb_records
    FROM RawRecords
    INNER JOIN Locations ON Locations.id = RawRecords.id_location
    WHERE id_location IN (SELECT id FROM Locations)
    GROUP BY id_location, date(date_timestamp) 
    ORDER BY date(date_timestamp);""",
    con=db_connection
)
pdata = pd.DataFrame(raw_data)
pdata.tail(100)

,dte,name,nb_records
1070,2022-03-08,Hérémence,95
1071,2022-03-08,Sion,92
1072,2022-03-08,Geneva,95
1073,2022-03-09,Lausanne,91
1074,2022-03-09,Sion,89
...,...,...,...
1165,2022-03-27,Geneva,83
1166,2022-03-27,Lausanne,86
1167,2022-03-27,Sion,84
1168,2022-03-28,Evolène,1


In [ ]:
#----------------------------------------------------------------------------------
# Table : RawRecords
# Query : List records created by locations and creation dates grouped 0
#         by locations + month
#----------------------------------------------------------------------------------

raw_data = pd.read_sql(
""" SELECT date(date_timestamp) AS dte, Locations.name AS LName, count(date_timestamp) AS nb_records
    FROM RawRecords
    INNER JOIN Locations ON Locations.id = RawRecords.id_location
    WHERE MONTH(DATE(date_timestamp)) in (SELECT MONTH(DATE(date_timestamp)) FROM RawRecords GROUP BY MONTH(DATE(date_timestamp)))
    AND ( id_location IN (SELECT id FROM Locations) ) 
    GROUP BY id_location, month(date_timestamp) 
    ORDER BY id_location, month(date_timestamp);""",
    con=db_connection
)
pdata = pd.DataFrame(raw_data)
pdata.tail(100)

In [7]:
#----------------------------------------------------------------------------------
# Table : RawRecords
# Select MIN,MAX,AVG by locations and creation dates grouped by locations + day
#----------------------------------------------------------------------------------
raw_data = pd.read_sql(
""" SELECT date(date_timestamp) AS dte, Locations.name AS LName, 
    (round(min(temp),2) - 273.15) as minTemp, (round(max(temp),2) - 273.15) as maxTemp, (round(avg(temp), 2) - 273.15) as avgTemp,
    round(min(humidity),2) as minHumidity, round(max(humidity),2) as maxHumidity, round(avg(humidity), 2) as avgHumidity,
    round(min(pressure),2) as minPressure, round(max(pressure),2) as maxPressure, round(avg(pressure), 2) as avgPressure,
    round(sum(rain_1h))
    FROM RawRecords
    INNER JOIN Locations ON Locations.id = RawRecords.id_location
    WHERE id_location IN (SELECT id FROM Locations)
    GROUP BY id_location, month(date_timestamp),day(date_timestamp)
    ORDER BY id_location, date_timestamp; """,
    con=db_connection
)
pdata = pd.DataFrame(raw_data)
pdata.tail(100)

,dte,LName,minTemp,maxTemp,avgTemp,minHumidity,maxHumidity,avgHumidity,minPressure,maxPressure,avgPressure,round(sum(rain_1h))
1070,2021-12-19,Hérémence,-9.46,5.79,-4.87,66.0,100.0,86.71,1024.0,1035.0,1028.89,0.0
1071,2021-12-20,Hérémence,-10.88,6.56,-4.90,68.0,93.0,87.67,1017.0,1025.0,1022.26,0.0
1072,2021-12-21,Hérémence,-9.05,2.41,-5.33,64.0,100.0,85.45,1018.0,1028.0,1021.72,0.0
1073,2021-12-22,Hérémence,-10.56,5.21,-4.58,59.0,92.0,70.88,1020.0,1028.0,1024.51,0.0
1074,2021-12-23,Hérémence,-10.81,4.22,-3.46,48.0,86.0,72.69,1017.0,1025.0,1020.48,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
1165,2022-03-24,Hérémence,-2.29,14.32,6.05,13.0,60.0,36.28,1023.0,1034.0,1026.84,0.0
1166,2022-03-25,Hérémence,-0.66,14.66,6.54,18.0,60.0,39.14,1020.0,1027.0,1024.64,0.0
1167,2022-03-26,Hérémence,-0.33,15.74,7.45,11.0,75.0,32.67,1022.0,1027.0,1024.72,0.0
1168,2022-03-27,Hérémence,-0.43,15.07,6.45,10.0,71.0,35.41,1024.0,1029.0,1027.06,0.0


In [ ]:
#----------------------------------------------------------------------------------
# Table : RawRecords
# Select MIN,MAX,AVG by locations and creation dates grouped by locations + month
#----------------------------------------------------------------------------------
raw_data = pd.read_sql(
""" SELECT date(date_timestamp) AS dte, Locations.name AS LName, 
    (round(min(temp),2) - 273.15) as minTemp, (round(max(temp),2) - 273.15) as maxTemp, (round(avg(temp), 2) - 273.15) as avgTemp,
    round(min(humidity),2) as minHumidity, round(max(humidity),2) as maxHumidity, round(avg(humidity), 2) as avgHumidity,
    round(min(pressure),2) as minPressure, round(max(pressure),2) as maxPressure, round(avg(pressure), 2) as avgPressure
    FROM RawRecords
    INNER JOIN Locations ON Locations.id = RawRecords.id_location
    WHERE MONTH(DATE(date_timestamp)) in (SELECT MONTH(DATE(date_timestamp)) FROM RawRecords GROUP BY MONTH(DATE(date_timestamp)))
    AND ( id_location IN (SELECT id FROM Locations) ) 
    GROUP BY id_location, month(date_timestamp) 
    ORDER BY id_location, month(date_timestamp);""",
    con=db_connection
)
pdata = pd.DataFrame(raw_data)
pdata.head(100)

In [ ]:
#----------------------------------------------------------------------------------
# Table : RawRecords
# Query : Count frozen days this year by location(s)
#----------------------------------------------------------------------------------
raw_data = pd.read_sql(
    """
        SELECT LocationName, count(date)
        FROM (
                SELECT date(date_timestamp) AS date, Locations.name AS LocationName
                FROM RawRecords
                INNER JOIN Locations ON Locations.id = RawRecords.id_location
                WHERE (id_location ) 
                GROUP BY id_location, MONTH(date_timestamp), DAY(date_timestamp)
                HAVING ( (round(max(temp), 2) - 273.15) < 0)
        ) tmpTable
        GROUP BY tmpTable.LocationName
        ;
    """,
    con=db_connection
)

pdata = pd.DataFrame(raw_data)
pdata.head(100)


In [ ]:
#----------------------------------------------------------------------------------
# Table : RawRecords
# Query : Count cold days (below 10°C) this year by location(s)
#----------------------------------------------------------------------------------

raw_data =  pd.read_sql(
    """
        SELECT LocationName, count(date)
        FROM (
                SELECT date(date_timestamp) AS date, Locations.name AS LocationName
                FROM RawRecords
                INNER JOIN Locations ON Locations.id = RawRecords.id_location
                WHERE (id_location ) 
                GROUP BY id_location, MONTH(date_timestamp), DAY(date_timestamp)
                HAVING ( (round(max(temp), 2) - 273.15) < 10)
        ) tmpTable
        GROUP BY tmpTable.LocationName
        ;
    """,
    con=db_connection
)

pdata = pd.DataFrame(raw_data)
pdata.head(100)

In [ ]:
#----------------------------------------------------------------------------------
# Table : RawRecords
# Query : Count hot days (over 20°C including night) this year by location(s)
#----------------------------------------------------------------------------------
raw_data = pd.read_sql(
    """
        SELECT LocationName, count(date)
        FROM (
                SELECT date(date_timestamp) AS date, Locations.name AS LocationName
                FROM RawRecords
                INNER JOIN Locations ON Locations.id = RawRecords.id_location
                WHERE (id_location ) 
                GROUP BY id_location, MONTH(date_timestamp), DAY(date_timestamp)
                HAVING ( (round(min(temp), 2) - 273.15) >= 20)
        ) tmpTable
        GROUP BY tmpTable.LocationName
        ;
    """,
    con=db_connection
)

pdata = pd.DataFrame(raw_data)
pdata.head(100)

In [ ]:
#----------------------------------------------------------------------------------
# Table : RawRecords
# Query : Count snow days  this year by location(s) anmd date
#----------------------------------------------------------------------------------
raw_data = pd.read_sql(
    """
        SELECT LocationName, count(date) AS snowDays
        FROM (
                SELECT date(date_timestamp) AS date, Locations.name AS LocationName, round((temp - 273.15),2) AS t
                FROM RawRecords
                INNER JOIN Locations ON Locations.id = RawRecords.id_location
                WHERE (id_location AND rain_1h > 0 ) 
                GROUP BY id_location, YEAR(date_timestamp), MONTH(date_timestamp), DAY(date_timestamp)
                HAVING ( t <= 2)
        ) tmpTable
        GROUP BY tmpTable.LocationName
        ;
    """,
    con=db_connection
)

pdata = pd.DataFrame(raw_data)
pdata.head(100)

In [8]:
#----------------------------------------------------------------------------------
# Table : RawRecords
# Query : Count rain/snow consecutive days and return a list of periods
#----------------------------------------------------------------------------------
import datetime
raw_data = pd.read_sql(
    """
        SELECT date(date_timestamp) AS date, Locations.name AS LocationName, round(avg(rain_1h) * 24,2) AS rain
        FROM RawRecords
        INNER JOIN Locations ON Locations.id = RawRecords.id_location
        WHERE (id_location = 6) AND (rain_1h > 0 ) 
        GROUP BY id_location, MONTH(date_timestamp), DAY(date_timestamp)
        HAVING ( rain > 0)
    """,
    con=db_connection
)

pdata = pd.DataFrame(raw_data)
tmpList = []
patternList = []
lastDate = datetime.datetime(1970,1,1)

for item in pdata.iterrows():
        if (item[1]['date'] != (lastDate + datetime.timedelta(days=1))):
            patternList.append(tmpList)
            tmpList = []        
            tmpList.append(item[1]) 
        lastDate = item[1]['date']
